In [16]:
#import packages
import pandas as pd

In [17]:
game_data = pd.read_csv('data/games.csv')

#remove 999usd game
i = game_data[(game_data.AppID == 1200520)].index
game_data.drop(i[0], inplace=True)

game_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70209 entries, 0 to 70209
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       70209 non-null  int64  
 1   Name                        70208 non-null  object 
 2   Release date                70209 non-null  object 
 3   Estimated owners            70209 non-null  object 
 4   Peak CCU                    70209 non-null  int64  
 5   Required age                70209 non-null  int64  
 6   Price                       70209 non-null  float64
 7   DLC count                   70209 non-null  int64  
 8   About the game              67908 non-null  object 
 9   Supported languages         70209 non-null  object 
 10  Full audio languages        70209 non-null  object 
 11  Reviews                     9121 non-null   object 
 12  Header image                70209 non-null  object 
 13  Website                     346

### Remove unnecessary columns (not relevant to data analysis) <br>
URLs/Contact info: website, support url, support email, metacritic url <br>
Images/videos: screenshots, movies, header image <br>
irrelevant/unusable data: user score(mostly NaN), score rank (mostly NULL), no. of achivements <br>
<br>
remove utilites: not games

In [18]:
removed_columns = ["Header image", "Website", "Support url", "Support email", "Metacritic url", "User score", "Score rank", "Achievements", "Screenshots", "Movies"]
game_data.drop(columns=removed_columns, inplace=True)
game_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70209 entries, 0 to 70209
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       70209 non-null  int64  
 1   Name                        70208 non-null  object 
 2   Release date                70209 non-null  object 
 3   Estimated owners            70209 non-null  object 
 4   Peak CCU                    70209 non-null  int64  
 5   Required age                70209 non-null  int64  
 6   Price                       70209 non-null  float64
 7   DLC count                   70209 non-null  int64  
 8   About the game              67908 non-null  object 
 9   Supported languages         70209 non-null  object 
 10  Full audio languages        70209 non-null  object 
 11  Reviews                     9121 non-null   object 
 12  Windows                     70209 non-null  bool   
 13  Mac                         702

In [19]:
#remove all utilities
game_data = game_data.loc[~game_data['Genres'].str.contains(r"utilities", case=False, na=False)]
game_data = game_data.loc[~game_data['Tags'].str.contains(r"utilities", case=False, na=False)]
game_data = game_data.loc[~game_data['Categories'].str.contains(r"utilities", case=False, na=False)]

Cleaning Genres

In [20]:
regex = r"Casual|Indie|Sports|Action|Adventure|Strategy|Massively Multiplayer|RPG|Simulation|Racing"
clean_table = game_data.loc[game_data['Genres'].str.contains(regex, case=False, na=False)]
# clean_gen.to_csv("gen.csv", sep=',', encoding='utf-8')
clean_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66417 entries, 0 to 70209
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       66417 non-null  int64  
 1   Name                        66417 non-null  object 
 2   Release date                66417 non-null  object 
 3   Estimated owners            66417 non-null  object 
 4   Peak CCU                    66417 non-null  int64  
 5   Required age                66417 non-null  int64  
 6   Price                       66417 non-null  float64
 7   DLC count                   66417 non-null  int64  
 8   About the game              66340 non-null  object 
 9   Supported languages         66417 non-null  object 
 10  Full audio languages        66417 non-null  object 
 11  Reviews                     8982 non-null   object 
 12  Windows                     66417 non-null  bool   
 13  Mac                         664

Limit dataset to games with more than x reviews

In [21]:
rev_table = pd.DataFrame(clean_table)
rev_table['Rate_Ratio'] = rev_table['Positive']/ (rev_table['Negative'] + rev_table['Positive'])
rev_table['Reviews'] = rev_table['Negative'] + rev_table['Positive']

rev_table.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Rate_Ratio
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",0.352941
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",0.913793
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,NaN
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",1.000000
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",0.862069


In [22]:
rev_table.dropna(axis=0, subset=["Rate_Ratio"], inplace=True)
small_review_price = rev_table[rev_table['Reviews'] >= 75]

In [23]:
small_review_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18142 entries, 5 to 70189
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       18142 non-null  int64  
 1   Name                        18142 non-null  object 
 2   Release date                18142 non-null  object 
 3   Estimated owners            18142 non-null  object 
 4   Peak CCU                    18142 non-null  int64  
 5   Required age                18142 non-null  int64  
 6   Price                       18142 non-null  float64
 7   DLC count                   18142 non-null  int64  
 8   About the game              18117 non-null  object 
 9   Supported languages         18142 non-null  object 
 10  Full audio languages        18142 non-null  object 
 11  Reviews                     18142 non-null  int64  
 12  Windows                     18142 non-null  bool   
 13  Mac                         181

Check for unneeded data

In [24]:
comp = "Categories"
genre_price = pd.DataFrame(small_review_price[["AppID", "Name", "Price", comp]])
genre = pd.DataFrame(game_data[comp])
u_gen = genre_price[comp].unique()
print(genre_price[comp].describe())
print(len(u_gen))
gen_list = []
for i in u_gen:
    try:
        arr = i.split(",")
    except:
        print(i)
    for j in arr:
        if j not in gen_list:
            gen_list.append(j)

print(gen_list)
print(len(gen_list))

count             18128
unique             3433
top       Single-player
freq               2096
Name: Categories, dtype: object
3434
nan
['Single-player', 'Multi-player', 'MMO', 'PvP', 'Online PvP', 'Co-op', 'Online Co-op', 'In-App Purchases', 'Steam Achievements', 'Full controller support', 'Steam Trading Cards', 'Partial Controller Support', 'Steam Cloud', 'Remote Play on Tablet', 'Shared/Split Screen', 'Cross-Platform Multiplayer', 'Remote Play Together', 'Stats', 'Steam Leaderboards', 'Remote Play on Phone', 'Shared/Split Screen PvP', 'Captions available', 'Steam Workshop', 'Includes level editor', 'LAN Co-op', 'LAN PvP', 'Remote Play on TV', 'Shared/Split Screen Co-op', 'Steam Turn Notifications', 'VR Support', 'SteamVR Collectibles', 'Valve Anti-Cheat enabled', 'Commentary available', 'Includes Source SDK', 'Mods (require HL2)', 'Mods', 'Tracked Controller Support', 'VR Only']
38


In [25]:
#create csv for cleaned data
small_review_price.reset_index(inplace=True, drop=True)
small_review_price.to_csv("data/clean_data.csv", sep=',', encoding='utf-8', index=False)

PermissionError: [Errno 13] Permission denied: 'data/clean_data.csv'